### FLEXISAF INTERNSHIP
### TASK 3 - NLP 

In [30]:
import numpy as np
import pandas as pd
import os
import glob
import re
import nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import string
pd.options.mode.chained_assignment = None

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Opening the File

In [31]:
file_path = r"c:/Users/DELL/Desktop/Flexisaf//sample.csv"
if not os.path.exists(file_path):
    print ("File not found, check the path.")
else:
    try:
        encodings = ["utf-8", "latin-1", "ISO-8859-1"]
        delimiters = [",", ";", "\t", "|"]
        
        df = None 
        for enc in encodings:
            for delim in delimiters:
                try:
                    df=pd.read_csv(file_path, encoding=enc, delimiter=delim)
                    print (f" file opened successfully with encoding='{enc}' and delimiter='{delim}'")
                    print(df.head())

                    break
                except Exception:
                    continue
            if df is not None:
                break
        if df is None:
            print("Could not read the CSV with common encodings/delimiters.")
    except Exception as e:
        print (f"Error: {e}")
        
            

 file opened successfully with encoding='utf-8' and delimiter=','
   tweet_id     author_id  inbound                      created_at  \
0    119237        105834     True  Wed Oct 11 06:55:44 +0000 2017   
1    119238  ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2    119239        105835     True  Wed Oct 11 13:00:09 +0000 2017   
3    119240  VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4    119241        105836     True  Tue Oct 10 15:17:21 +0000 2017   

                                                text response_tweet_id  \
0  @AppleSupport causing the reply to be disregar...            119236   
1  @105835 Your business means a lot to us. Pleas...               NaN   
2  @76328 I really hope you all change but I'm su...            119238   
3  @105836 LiveChat is online at the moment - htt...            119241   
4  @VirginTrains see attached error message. I've...            119243   

   in_response_to_tweet_id  
0                      NaN  
1         

In [32]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tweet_id                 93 non-null     int64  
 1   author_id                93 non-null     object 
 2   inbound                  93 non-null     bool   
 3   created_at               93 non-null     object 
 4   text                     93 non-null     object 
 5   response_tweet_id        65 non-null     object 
 6   in_response_to_tweet_id  68 non-null     float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 4.6+ KB


### 1. Lower Casing

In [34]:
# creating a new column for the text lower case
df["text_lower_case"] = df["text"].str.lower()

# drop the  original text column given
df.drop(["text"], axis=1, inplace=True)
df.head()

,tweet_id,author_id,inbound,created_at,response_tweet_id,in_response_to_tweet_id,text_lower_case
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,119236,NaN,@applesupport causing the reply to be disregar...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,NaN,119239.0,@105835 your business means a lot to us. pleas...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,119238,NaN,@76328 i really hope you all change but i'm su...
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,119241,119242.0,@105836 livechat is online at the moment - htt...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,119243,119240.0,@virgintrains see attached error message. i've...


### 2. Removal of Punctuation

In [35]:
# creating a function to remove the punctuation mark using string punctuation 
rem_punct = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', rem_punct))
df["text_rem_punct"] = df["text_lower_case"].apply(lambda text: remove_punctuation(text))

# drop the previously modified column
df.drop(["text_lower_case"], axis=1, inplace=True)
df.head()

,tweet_id,author_id,inbound,created_at,response_tweet_id,in_response_to_tweet_id,text_rem_punct
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,119236,NaN,applesupport causing the reply to be disregard...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,NaN,119239.0,105835 your business means a lot to us please ...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,119238,NaN,76328 i really hope you all change but im sure...
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,119241,119242.0,105836 livechat is online at the moment https...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,119243,119240.0,virgintrains see attached error message ive tr...


### 3.  Removal of frequent words

In [36]:
# Removing the stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stop_words])

df["text_rem_stop"] = df["text_rem_punct"].apply(lambda text: remove_stopwords(text))

# drop the previously modified column
df.drop(["text_rem_punct"], axis=1, inplace=True)

df.head()

,tweet_id,author_id,inbound,created_at,response_tweet_id,in_response_to_tweet_id,text_rem_stop
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,119236,NaN,applesupport causing reply disregarded tapped ...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,NaN,119239.0,105835 business means lot us please dm name zi...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,119238,NaN,76328 really hope change im sure wont dont
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,119241,119242.0,105836 livechat online moment httpstcosy94vtu8...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,119243,119240.0,virgintrains see attached error message ive tr...


In [37]:
# counting the modals of the words  
from collections import Counter
cnt = Counter()
for text in df["text_rem_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(20)

[('us', 25),
 ('dm', 19),
 ('help', 18),
 ('thanks', 13),
 ('httpstcogdrqu22ypt', 12),
 ('applesupport', 11),
 ('please', 11),
 ('phone', 9),
 ('hi', 9),
 ('ive', 8),
 ('version', 8),
 ('get', 8),
 ('tesco', 8),
 ('device', 7),
 ('ios', 7),
 ('look', 7),
 ('76099', 7),
 ('let', 7),
 ('using', 7),
 ('spotifycares', 7)]

In [38]:
# creating a function for frequent words
Freqword = set([w for (w, wc) in cnt.most_common(20)])
def remove_freqwords(text):
    """function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in Freqword])

df["text_rem_freq"] = df["text_rem_stop"].apply(lambda text: remove_freqwords(text))

# drop the previously modified text column
df.drop(["text_rem_stop"], axis=1, inplace=True)
df.head(10)


,tweet_id,author_id,inbound,created_at,response_tweet_id,in_response_to_tweet_id,text_rem_freq
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,119236,NaN,causing reply disregarded tapped notification ...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,NaN,119239.0,105835 business means lot name zip code additi...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,119238,NaN,76328 really hope change im sure wont dont
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,119241,119242.0,105836 livechat online moment httpstcosy94vtu8...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,119243,119240.0,virgintrains see attached error message tried ...
5,119243,VirginTrains,False,Tue Oct 10 15:25:14 +0000 2017,119244,119241.0,105836 tried another miriam mm
6,119244,105836,True,Tue Oct 10 15:26:44 +0000 2017,119245,119243.0,virgintrains yep tried laptop several times pa...
7,119245,VirginTrains,False,Tue Oct 10 15:33:22 +0000 2017,NaN,119244.0,105836 working ok miriam link httpstco0m2mph15...
8,119242,105836,True,Tue Oct 10 15:09:00 +0000 2017,119240,119246.0,virgintrains still havent heard amp number im ...
9,119246,VirginTrains,False,Tue Oct 10 10:13:19 +0000 2017,119242,119247.0,105836 thats miriam 😊 team send email shortly hp


### 4. Removal of Rare words

In [39]:
n_rare_words = 10
rare_words = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in rare_words])

df["text_rem_rare"] = df["text_rem_freq"].apply(lambda text: remove_rarewords(text))

# drop the previously modified text column
df.drop(["text_rem_freq"], axis=1, inplace=True)
df.head()

,tweet_id,author_id,inbound,created_at,response_tweet_id,in_response_to_tweet_id,text_rem_rare
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,119236,NaN,causing reply disregarded tapped notification ...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,NaN,119239.0,105835 business means lot name zip code additi...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,119238,NaN,76328 really hope change im sure wont dont
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,119241,119242.0,105836 livechat online moment httpstcosy94vtu8...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,119243,119240.0,virgintrains see attached error message tried ...
